In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
%matplotlib inline
w=np.array(np.random.rand(1000)*1)
values=np.array(np.random.rand(1000)*20)
indices=values.argsort()
w=w[indices]
values=values[indices]
#print(values)
bins=np.linspace(values.min(), values.max(),20)
print(bins)
mid_bins=0.5*(bins[:-1]+bins[1:])
batch_indx=np.asarray([[bins[i],bins[i+1] ]for i in range(len(bins)-1)])

batches=[values[(values>=lbound) & (values<hbound)] for lbound,hbound in batch_indx]
weights=[w[(values>=lbound) & (values<hbound)] for lbound,hbound in batch_indx]
s_weights=[np.sum(w_b)/len(w_b) for w_b in weights]
s_weights=np.asarray(s_weights)
cs=np.cumsum(indices)

width = 1

clist = [(0, "red"),(s_weights.min(), "red"), (s_weights.max(), "blue"),(1, "blue")]
rvb = mcolors.LinearSegmentedColormap.from_list("", clist)

ax = plt.subplot(111)
ax.bar(mid_bins, s_weights, width, color=rvb(s_weights))
plt.show()

In [1]:
import matplotlib.pyplot as plt
params=['nphotons', 'mpi_np', 'mus','mua','musp','collection_radius', 'slab_sizes','nangles', 'NNr', 'NNxy', 'NNz', 'dr', 'dxdx', 'dz', 'no','ne', 'chi', 'lambda','birefringence directional cosine 0','b1','b2', ]
with open('parameters.txt', 'r') as f:
    #for i, line in enumerate(f.readline()):
    i=0
    for line in f:
        print(params[i],line.strip())
        i+=1

nphotons 1000000
mpi_np 4
mus 100.00076
mua 0.00000
musp 100.00000
collection_radius 0.05000
slab_sizes 1.00000
nangles 10001
NNr 501
NNxy 1001
NNz 501
dr 0.00009980039920159681
dxdx 0.00009990009990009990
dz 0.00009980039920159681
no 1.3300000000
ne 1.3300000000
chi 0.0000000000
lambda 0.6330000000
birefringence directional cosine 0 1.0000000000
b1 0.0000000000
b2 0.0000000000


In [ ]:
real=[]
imag=[]
real=[]
imag=[]
with open('s2.txt','r') as f:
    i=0
    for line in f:
        real.append(float(line.strip().split( )[0]))
        imag.append(float(line.strip().split( )[1]))

ax0=plt.subplot(1,2,1)
ax0.plot(real)
ax1=plt.subplot(1,2,2)
ax1.plot(imag)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
real=[]
imag=[]
real=[]
imag=[]
with open('m11.txt','r') as f:
    i=0
    for line in f:
        real.append(float(line.strip().split( )[0]))
        #imag.append(float(line.strip().split( )[1]))

ax0=plt.subplot(1,2,1)
ax0.plot(real)
ax1=plt.subplot(1,2,2)
ax1.plot(imag)
plt.show()

In [ ]:
import numpy as np
from numpy import genfromtxt

my_data = genfromtxt('s1_pm_1000.txt', delimiter=',')
my_data
plt.plot(my_data)
plt.show()

res=1j*my_data[:,2]
res+=my_data[:,1]
#print(my_data[:,1].shape)
#np.complex(my_data[:,1],my_data[:,2])
res

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.ticker as plticker
%matplotlib inline
%load_ext autoreload
%autoreload 2

rads=np.linspace(0,np.pi*2,720)
xticks=list(np.linspace(0,720,7, dtype=int))
#print(xticks)
new_ticks=list(np.linspace(0,360,7, dtype=int))
gs=[0.5,0.9,0.99]
f=np.zeros((3,720))
for i,g in enumerate(gs):
    f[i,:]=(1-g*g)/(2*(1+g*g-2*g*np.cos(rads)))
    
# fig, ax = plt.subplots()
# ax.plot(f[0].T)
# ax.set_xticklabels([0,1,2], [4,5,6])
# loc = plticker.MultipleLocator(base=32) # this locator puts ticks at regular intervals
#ax.xaxis.set_major_locator(loc)
#ax.xaxis.set_ticks(xticks)
#ax.xaxis.set_major_formatter(ticker.FormatStrFormatter('%0.1f'))
#plt.xticks(xticks)
# ax.set_ylim((0,10))
# plt.show()
plt.plot(rads,f[0],'r-')
plt.plot(rads,f[1], '--b')
plt.plot(rads,f[2], '--k')
plt.legend([f'g = {str(gs[0])}',f'g = {str(gs[1])}',f'g = {str(gs[2])}'])
#plt.xticks(xticks,new_ticks)
plt.xlabel('Radian', fontsize=12)
plt.savefig('gs.png')
plt.show()

In [ ]:
from photon import *
import numpy as np
from numba import typed, types
import numba as nb
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from plot import *
from numpy import genfromtxt
from numba.cuda.random import init_xoroshiro128p_states

gridDim = 515
blockDim = 512
dim = gridDim * blockDim
all_adj_dist = []
all_collected = []
all_amps = []
all_modes = []
mode = 1.0
iters = 3 * 16
#iters = 1
NNxy = 100
NNr = 50
NNz = 50
degree_divs = 1000

s1 = genfromtxt('s1_pm_1000.txt', delimiter=',')
s2 = genfromtxt('s2_pm_1000.txt', delimiter=',')
m11 = genfromtxt('m11_pm_1000.txt', delimiter=',')
m12 = genfromtxt('m12_pm_1000.txt', delimiter=',')
temp = 1j * s1[:, 2]
temp += s1[:, 1]
s1 = temp
temp = 1j * s2[:, 2]
temp += s2[:, 1]
s2 = temp
temp = 1j * m11[:, 2]
temp += m11[:, 1]
m11 = temp
temp = 1j * m12[:, 2]
temp += m12[:, 1]
m12 = temp
#print(m11[1])
print(cuda.gpus)
for i in range(iters):
    mode = i % 3
    #mode = 3

    amplitudes = np.ones(dim, dtype=np.float32)
    steps = np.zeros(dim, dtype=np.float32)
    lengths = np.zeros(dim, dtype=np.float32)
    direction_theta = np.ones(dim, dtype=np.float32) * (np.pi / 2)
    maxZs = np.zeros(dim, dtype=np.float32)
    position_x = np.zeros(dim, dtype=np.float32)
    position_y = np.ones(dim, dtype=np.float32) * (1e-8)
    polarization = np.ones(dim, dtype=np.float32)
    adjusted_dist = np.zeros(dim, dtype=np.float32)
    collected = np.zeros(dim, dtype=np.float32)
    modes = np.ones(dim, dtype=np.float32) * mode  # polarization mode 0:nonpolarized 1:p-polarized 2:s-polarized

    jones = np.zeros((dim, 4), dtype=np.complex64)
    #jones=cuda.to_device(jones)
    jones_partial = np.zeros((dim, 4), dtype=np.complex64)
    U = np.zeros((dim, 3), dtype=np.float32)
    W = np.zeros(dim, dtype=np.float32)
    mu_as = np.ones(dim, dtype=np.float32)*0
    mu_ss = np.ones(dim, dtype=np.float32)*1000
    scat_events = np.zeros(dim, dtype=np.float32)

    co_xy = np.zeros((NNxy, NNxy), dtype=np.float32)
    cross_xy = np.zeros((NNxy, NNxy), dtype=np.float32)
    incoh_cross_xy = np.zeros((NNxy, NNxy), dtype=np.float32)
    co_rz = np.zeros((NNr, NNz), dtype=np.float32)
    cross_rz = np.zeros((NNr, NNz), dtype=np.float32)
    incoh_cross_rz = np.zeros((NNr, NNz), dtype=np.float32)

    co_xy_trad = np.zeros((NNxy, NNxy), dtype=np.float32)
    cross_xy_trad = np.zeros((NNxy, NNxy), dtype=np.float32)
    incoh_cross_xy_trad = np.zeros((NNxy, NNxy), dtype=np.float32)
    co_rz_trad = np.zeros((NNr, NNz), dtype=np.float32)
    cross_rz_trad = np.zeros((NNr, NNz), dtype=np.float32)
    incoh_cross_rz_trad = np.zeros((NNr, NNz), dtype=np.float32)

    position = np.zeros((NNxy, 3), dtype=np.float32)


    rng_states = create_xoroshiro128p_states((gridDim * blockDim)*300 , seed=int(mode + 10 + i))
    #print(rng_states)
    #rng_states = init_xoroshiro128p_states(gridDim * blockDim + i + int(mode), seed=int(mode + 1 + i))
    process_steps[gridDim, blockDim](amplitudes, direction_theta, position_x, position_y, polarization, steps, lengths,
                                     maxZs, rng_states, adjusted_dist, collected, modes, U, W, jones, mu_as, mu_ss,
                                     scat_events, jones_partial,co_xy, cross_xy, incoh_cross_xy, co_rz, cross_rz, incoh_cross_rz,
                                     co_xy_trad, cross_xy_trad, incoh_cross_xy_trad, co_rz_trad, cross_rz_trad,
                                     incoh_cross_rz_trad, position, s1, s2,m11,m12)
    #nb.cuda.profile_stop()

    all_adj_dist.append(adjusted_dist)
    all_collected.append(collected)
    all_amps.append(amplitudes)
    all_modes.append(modes)


In [ ]:
all_adj_dist = np.asarray(all_adj_dist)
all_collected = np.asarray(all_collected)
all_amps = np.asarray(all_amps)
all_adj_dist = all_adj_dist.reshape([-1])
all_collected = all_collected.reshape([-1])
all_amps = all_amps.reshape([-1])
all_modes = np.asarray(all_modes)
all_modes = all_modes.reshape([-1])

position_y_cs = np.cumsum(position_y)
weighted = position_y * amplitudes
weighted_cs = np.cumsum(weighted)

# print(all_collected.shape)
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
fig.tight_layout()
photon_type = ['Non Polarized', 'P-Polarized', 'S-Polarized']
#print(all_modes ==0)
print(len(all_adj_dist[(all_collected > 0) & (all_modes==0)] ))
# print(len(all_adj_dist[(all_collected > 0) & (all_modes==1)] ))
# print(len(all_adj_dist[(all_collected > 0) & (all_modes==2)] ))
print(len(all_adj_dist[(all_collected > 0) & (all_modes == 2)]) / len(all_adj_dist[all_modes == 2]))
# print(len(axes[1]))
for i, ax in enumerate(axes[0]):
    if i == 3:
        break
    # n, bins, patches=ax.hist(all_adj_dist[(all_collected > 0) & (all_modes==i)], bins=500, density=True, log=True, weights=all_amps[(all_collected > 0) & (all_modes==i)] / (iters * dim / 3))
    ax = plot(all_amps[(all_collected > 0) & (all_modes == i)], all_adj_dist[(all_collected > 0) & (all_modes == i)],
              ax)
    ax.set_title(photon_type[i])
    # print(len(n))
    ax.set_xlim([0, 1.2e-3])
    # bin_centers = 0.5 * (bins[:-1] + bins[1:])
    # scale values to interval [0,1]
    # col = bin_centers - min(bin_centers)
    # col /= max(col)
    # cm = plt.cm.get_cmap('RdYlBu_r')
    # for c, p in zip(col, patches):
    #     plt.setp(p, 'facecolor', cm(c))

stoke_vectors = ['S_0', 'S_1', 'S_2', 'S_3']
for i, stoke_vector in enumerate(stoke_vectors):
    axes[1][i], s_0, frequency_bins = plot_stokes(all_amps[(all_collected > 0) & (all_modes == 1)],
                                                  all_adj_dist[(all_collected > 0) & (all_modes == 1)],
                                                  all_amps[(all_collected > 0) & (all_modes == 2)],
                                                  all_adj_dist[(all_collected > 0) & (all_modes == 2)], axes[1][i],
                                                  stokes=stoke_vector)
    axes[1][i].set_xlim([0, 1.2e-3])
    axes[1][i].set_title(stoke_vectors[i])

for ax in axes:
    for a in ax:
        for layer in y_array:
            a.axvline(layer, color='r', linestyle='dashed', linewidth=1)

plt.savefig('octsimplemp.png')

In [ ]:
fig, axes = plt.subplots(1, figsize=(16, 8))
fig.tight_layout()
ax=axes
ax = plot(all_amps[(all_collected > 0) & (all_modes == 0)], all_adj_dist[(all_collected > 0) & (all_modes == 0)],
          ax)
ax.set_title(photon_type[0])
# print(len(n))
ax.set_xlim([0, 1.2e-3])
for layer in y_array:
    ax.axvline(layer, color='r', linestyle='dashed', linewidth=1)
plt.savefig('octsimplemp.png')
fig.show()

In [ ]:
from numba.pycc import CC
from photon import *
import numpy as np
from numba import typed, types
import numba as nb
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from plot import *
from numpy import genfromtxt
from numba.cuda.random import init_xoroshiro128p_states
from mpl_toolkits.mplot3d import Axes3D
import os
from matplotlib import cm
from scipy import optimize
%matplotlib inline
%load_ext autoreload
%autoreload 2

def fit_func(x, const, a, b, c, d):
    return np.power(x, 3) * d + np.power(x, 2) * c + np.power(x, 2) * b + x * a + const

gridDim = 1
blockDim = 256
dim = gridDim * blockDim
all_adj_dist = []
all_collected = []
all_amps = []
all_modes = []
mode = 1.0
iters = 3 * 16
iters = 4
NNxy = 500
NNr = int(NNxy/2)
NNz = int(NNxy/2)
degree_divs = 64

s1 = genfromtxt('s1_pm_64.txt', delimiter=',')
s2 = genfromtxt('s2_pm_64.txt', delimiter=',')
m11 = genfromtxt('m11_pm_64.txt', delimiter=',')
m12 = genfromtxt('m12_pm_64.txt', delimiter=',')
temp = 1j * s1[:, 2]
temp += s1[:, 1]
s1 = temp
temp = 1j * s2[:, 2]
temp += s2[:, 1]
s2 = temp
m11 = m11[:, 1]
m12 = m12[:, 1]

s1 = np.ascontiguousarray(s1)
s2 = np.ascontiguousarray(s2)
m11 = np.ascontiguousarray(m11)
m12 = np.ascontiguousarray(m12)
cuda.pinned(s1)
cuda.pinned(s2)
cuda.pinned(m11)
cuda.pinned(m12)

co_xy_all = np.zeros((dim,NNxy, NNxy), dtype=np.float32)
incoh_cross_xy_all=np.zeros((dim,NNxy, NNxy), dtype=np.float32)
cross_xy_all=np.zeros((dim,NNxy, NNxy), dtype=np.float32)

co_rz_all = np.zeros((dim, NNr, NNz), dtype=np.float32)
incoh_cross_rz_all=np.zeros((dim, NNr, NNz), dtype=np.float32)
cross_rz_all=np.zeros((dim, NNr, NNz), dtype=np.float32)

co_rz_trad_all = np.zeros((dim, NNr, NNz), dtype=np.float32)
ctr=0
ctr_trad=0
ctr_cross=0
#os.environ['NUMBA_DEBUG']='1'
#print(cuda.gpus)
for i in range(iters):
    mode = i % 3
    mode = 3

    amplitudes = np.ones(dim, dtype=np.float32)
    steps = np.zeros(dim, dtype=np.float32)
    lengths = np.zeros(dim, dtype=np.float32)
    direction_theta = np.ones(dim, dtype=np.float32) * (np.pi / 2)
    maxZs = np.zeros(dim, dtype=np.float32)
    position_x = np.zeros(dim, dtype=np.float32)
    position_y = np.ones(dim, dtype=np.float32) * (1e-8)
    polarization = np.ones(dim, dtype=np.float32)
    adjusted_dist = np.zeros(dim, dtype=np.float32)
    collected = np.zeros(dim, dtype=np.float32)
    modes = np.ones(dim, dtype=np.float32) * mode  # polarization mode 0:nonpolarized 1:p-polarized 2:s-polarized

    jones = np.zeros((dim, 4), dtype=np.complex64)
    # jones=cuda.to_device(jones)
    jones_partial = np.zeros((dim, 4), dtype=np.complex64)
    U = np.zeros((dim, 3), dtype=np.float32)
    W = np.zeros(dim, dtype=np.float32)
    mu_as = np.ones(dim, dtype=np.float32) * 0
    mu_ss = np.ones(dim, dtype=np.float32) * 1000
    scat_events = np.zeros(dim, dtype=np.float32)

    co_xy = np.zeros((dim, NNxy, NNxy), dtype=np.float32)
    cross_xy = np.zeros((dim, NNxy, NNxy), dtype=np.float32)
    incoh_cross_xy = np.zeros((dim, NNxy, NNxy), dtype=np.float32)
    co_rz = np.zeros((dim, NNr, NNz), dtype=np.float32)
    cross_rz = np.zeros((dim, NNr, NNz), dtype=np.float32)
    incoh_cross_rz = np.zeros((dim, NNr, NNz), dtype=np.float32)

    co_xy_trad = np.zeros((dim, NNxy, NNxy), dtype=np.float32)
    cross_xy_trad = np.zeros((dim, NNxy, NNxy), dtype=np.float32)
    incoh_cross_xy_trad = np.zeros((dim, NNxy, NNxy), dtype=np.float32)
    co_rz_trad = np.zeros((dim, NNr, NNz), dtype=np.float32)
    cross_rz_trad = np.zeros((dim, NNr, NNz), dtype=np.float32)
    incoh_cross_rz_trad = np.zeros((dim, NNr, NNz), dtype=np.float32)

    position = np.zeros((NNxy, 3), dtype=np.float32)

    rng_states = create_xoroshiro128p_states(dim, seed=int(mode + 10 + i))
    seed = i*3
    print('iteration',i)
    # print(rng_states)
    # rng_states = init_xoroshiro128p_states(gridDim * blockDim + i + int(mode), seed=int(mode + 1 + i))
    process_steps[gridDim, blockDim](seed, amplitudes, direction_theta, position_x, position_y, polarization, steps,
                                     lengths,
                                     maxZs, rng_states, adjusted_dist, collected, modes, U, W, jones, mu_as, mu_ss,
                                     scat_events, jones_partial, co_xy, cross_xy, incoh_cross_xy, co_rz, cross_rz,
                                     incoh_cross_rz,
                                     co_xy_trad, cross_xy_trad, incoh_cross_xy_trad, co_rz_trad, cross_rz_trad,
                                     incoh_cross_rz_trad, position, s1, s2, m11, m12)
    # nb.cuda.profile_stop()

    all_adj_dist.append(adjusted_dist)
    all_collected.append(collected)
    all_amps.append(amplitudes)
    all_modes.append(modes)
    
    
    if not(np.isnan(co_rz.max())):
        co_rz_all += co_rz
        co_xy_all += co_xy  
        
        ctr+=1
        print(ctr, 'partial',co_rz.max())
    if not(np.isnan(co_rz_trad.max())):
        co_rz_trad_all+=co_rz_trad
        ctr_trad+=1
        print(ctr_trad, 'trad',co_rz_trad.max())
        
    if not (np.isnan(incoh_cross_xy_all.max())):
        incoh_cross_xy_all+=incoh_cross_xy        
        
        incoh_cross_rz_all+=incoh_cross_rz      
        ctr_cross+=1
    if not (np.isnan(cross_xy_all.max())):
        cross_xy_all+=cross_xy
        cross_rz_all+=cross_rz

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
%load_ext autoreload
%autoreload 2

grid_size = NNr
x = np.sum(co_rz_all, axis=0)
#print(np.unique(co_rz_all))
x[0,0]=0
x[grid_size-1,grid_size-1]=0
# mean = np.mean(x)
# std = np.std(x)
# x = (x - mean) / std
x/=ctr
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
X, Y = np.meshgrid(range(grid_size), range(grid_size))  # `plot_surface` expects `x` and `y` data to be 2D
ax.plot_surface(X, Y, x)
ax.set_zlim(0, 1)
plt.show()


In [ ]:
x_clipped=x/x.max()
x_clipped=np.clip(x,0,1)
plt.imshow(x_clipped)
plt.show()

In [ ]:
from scipy import optimize
def fit_func(x, const, a, b, c,d,e):
    return np.power(x,4)*e+np.power(x, 3) * d + np.power(x, 2) * c + np.power(x, 2) * b + x * a + const
pz=np.sum(x[:-1,:-1],axis=0)
pz=pz/np.sum(pz)
x_data = np.array(list(range(len(pz))))

params, params_covariance = optimize.curve_fit(fit_func, x_data, pz, p0=[0,0.01,0.01,0.01,0.01,0.01])
#plt.plot(x_data, fit_func(x_data, params[0], params[1], params[2], params[3], params[4], params[5]),label='Fitted function')

# for p in co_rz_all[:]:
#     p[0,0]=0
#     p[NNr-1,NNr-1]=0
#     q=np.mean(p,axis=0)
#     q=q/np.sum(q)
#     plt.plot(q,'o')
y=np.sum(co_rz_all,axis=0)
y[0,0]=0
y[NNr-1,NNr-1]=0
y=np.mean(y,axis=0)
y=y/np.sum(y)
plt.plot(y,'o')
#plt.ylim([0,0.1])
plt.show()

In [ ]:
x_normal=np.mean(x[:-1,:-1],axis=0)
x_normal=x_normal/x_normal.max()
x_normal/=np.sum(x_normal)
plt.plot(x_normal,'o')
plt.show()

In [ ]:
#x_normal=np.mean(x[:-1,:-1])
# x_normal=x[:-1,:-1]/x_normal.max()
# x_normal/=np.sum(x_normal)
# x_normal/=x_normal.max()
x_normal = np.clip((x-x.mean())*x.std(),0,1)
plt.imshow(x_normal[:NNr//3,:NNr//2],cmap=plt.get_cmap('viridis'))
plt.show()

In [ ]:
n=3
dx=5e-5
NNr = len(cross_rz[0,:,0])
x_data=np.array(range(NNr))*dx*1000
ret = np.cumsum(np.mean(x,axis=0), dtype=float)
ret[n:] = ret[n:] - ret[:-n]
ret=ret[n - 1:] / n
plt.plot(x_data[n-1:],ret/(NNr*dx*1000))
plt.show()
print(np.sum(x_data))
print(NNr*dx*1000)

In [ ]:
dx=5e-5

pz=np.sum(x[:-1,:-1],axis=0)
pz=pz/np.sum(pz)
x_data = np.array(list(range(len(pz))))
y=np.sum(co_rz_all,axis=0)
y[0,0]=0
y[NNr-1,NNr-1]=0
y=np.mean(y,axis=0)
y=y/np.sum(y)
y/=(dx*1000)
xs=np.array(range(len(y)))*(dx*1000)
plt.plot(xs,y,'o')
#plt.ylim([0,0.1])
plt.xlim([0,4])
plt.show()

In [ ]:
xx=np.sum(co_xy_all+cross_xy_all)
xy=np.sum(co_xy_all)-np.sum(cross_xy_all)
a=np.sum(co_xy_all,axis=0)
a[0,0]=0
a=np.sum(a)
b=np.sum(incoh_cross_xy_all,axis=0)
where_are_NaNs = np.isnan(b)
b[where_are_NaNs] = 0
where_are_NaNs = np.isnan(b)
b=np.sum(b)
#b=np.abs(b)
print((a+b)/a/2)

In [ ]:
c=np.sum(co_xy_all,axis=0)
d=np.sum(incoh_cross_xy_all,axis=0)
print(np.sum(d))

In [ ]:
l=1.465559
vv=1.453065
hv=1.553473
hp=2
oh=1.337506
vec=[l,vv,hv,hp,oh]
for el in vec:
    print(el/np.sum(vec))

In [ ]:
a1=np.sum(co_xy_all,axis=0)
a2=np.sum(a1[1:,1:])
b1=~np.isnan(cross_xy_all)
b1=np.sum(b1,axis=0)
b2=np.sum(b1[1:,1:])
sum=a2+b2+a1[0,0]+b1[0,0]
print(a1[0,0]/sum)